In [ ]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import itertools
import warnings

cf10

In [ ]:
df_path_cf10 = 'cf10/ww-df'

In [ ]:
def delta_merge_layers(folder_path,suffix):
    # Initialize dictionaries to store DataFrames for ft and non-ft files
    dataframes_ft = {}
    dataframes_non_ft = {}

    # List all CSV files in the folder
    files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

    # Loop through each file and categorize it as 'ft' or 'non-ft'
    for file in files:
        # Check if file matches the ft pattern
        if f'_{suffix}_ft.csv' in file:
            base_name = file.replace(f'_{suffix}_ft.csv', '')
            file_path = os.path.join(folder_path, file)
            # Read the entire DataFrame from the ft file
            df = pd.read_csv(file_path)
            if not df.empty:
                dataframes_ft[base_name] = df
        # Check if file matches the non-ft pattern
        elif f'_{suffix}.csv' in file:
            base_name = file.replace(f'_{suffix}.csv', '')
            file_path = os.path.join(folder_path, file)
            # Read the entire DataFrame from the non-ft file
            df = pd.read_csv(file_path)
            if not df.empty:
                dataframes_non_ft[base_name] = df

    # Initialize a list to store the differences DataFrames
    list_differences = []

    # Compute the differences for each matching pair
    for base_name in dataframes_ft:
        if base_name in dataframes_non_ft:
            df_ft = dataframes_ft[base_name]
            df_non_ft = dataframes_non_ft[base_name]

            # Align DataFrames on both axes (rows and columns)
            df_ft_aligned, df_non_ft_aligned = df_ft.align(df_non_ft, join='outer', axis=None, fill_value=0)

            # Convert boolean columns to integers
            for df in [df_ft_aligned, df_non_ft_aligned]:
                bool_cols = df.select_dtypes(include=['bool']).columns
                df[bool_cols] = df[bool_cols].astype(int)

            # Convert all columns to numeric, coercing errors to NaN
            df_ft_numeric = df_ft_aligned.apply(pd.to_numeric, errors='coerce')
            df_non_ft_numeric = df_non_ft_aligned.apply(pd.to_numeric, errors='coerce')

            # Fill NaN values with 0 (optional, depending on how you want to handle missing values)
            df_ft_numeric = df_ft_numeric.fillna(0)
            df_non_ft_numeric = df_non_ft_numeric.fillna(0)

            # Compute the difference between the ft and non-ft DataFrames
            df_diff = df_ft_numeric - df_non_ft_numeric

            # Add a column to identify the pair
            df_diff['base_name'] = base_name
            df_diff['layer_id'] = df_ft_numeric['layer_id']

            # Optional: Reset index to ensure a proper stacking
            df_diff.reset_index(drop=True, inplace=True)

            # Append to the list
            list_differences.append(df_diff)

    # Concatenate all the difference DataFrames
    if list_differences:
        df_all_differences = pd.concat(list_differences, ignore_index=True)
    else:
        df_all_differences = pd.DataFrame()  # Return empty DataFrame if no differences found

    return df_all_differences


In [ ]:
la_cf10 = delta_merge_layers(df_path_cf10, 'cf10')

In [ ]:
la_cf10.shape

In [ ]:
la_cf10 = la_cf10[['layer_id', 'alpha',  'entropy', 'log_norm', 'log_spectral_norm', 'base_name']]

In [ ]:
la_d_cf10 = la_cf10.drop(la_cf10[['alpha',  'entropy', 'log_norm', 'log_spectral_norm']][(la_cf10[['alpha',  'entropy', 'log_norm', 'log_spectral_norm']] == 0).all(axis=1)].index)

In [ ]:
def compute_pairwise_differences(df):
    """
    Computes pairwise differences between all categories and 'bs' based on 'base_name',
    for matching 'layer_id's.

    Parameters:
        df (pd.DataFrame): The input DataFrame containing 'base_name', 'layer_id', and numeric columns.

    Returns:
        pd.DataFrame: A DataFrame containing the differences with columns 'base_name', 'layer_id', and difference columns.
    """
    # Identify numeric columns excluding 'layer_id'
    numeric_cols = df.select_dtypes(include='number').columns.tolist()
    if 'layer_id' in numeric_cols:
        numeric_cols.remove('layer_id')

    # Separate 'bs' and other categories
    df_bs = df[df['base_name'] == 'bs']
    df_others = df[df['base_name'] != 'bs']

    # Merge on 'layer_id'
    df_merged = pd.merge(
        df_others,
        df_bs,
        on='layer_id',
        suffixes=('', '_bs'),
        how='inner'  # Ensure only matching 'layer_id's are joined
    )

    # Compute differences for numeric columns
    for col in numeric_cols:
        df_merged[f'{col}_diff'] = df_merged[col] - df_merged[f'{col}_bs']

    # Prepare the final DataFrame
    cols_to_keep = ['base_name', 'layer_id'] + [f'{col}_diff' for col in numeric_cols]
    df_differences = df_merged[cols_to_keep]

    # Reset index and sort (optional)
    df_differences.reset_index(drop=True, inplace=True)
    df_differences.sort_values(by=['base_name', 'layer_id'], inplace=True)

    return df_differences

In [ ]:
la_dif_cf10 = compute_pairwise_differences(la_d_cf10)

In [ ]:
def categorize_index(index_value):
    if 'div' in index_value:
        return 'div'
    elif 'dp' in index_value:
        return 'dp'
    elif 'wd' in index_value:
        return 'wd'
    else:
        return 'bs'  # Optional: in case none of the patterns match

# Apply the function to the index and create a new column
la_dif_cf10['category'] = la_dif_cf10['base_name'].map(categorize_index)

In [ ]:
la_avg_cf10 = la_dif_cf10.drop(['base_name'], axis=1).groupby(['layer_id', 'category'], as_index=False).mean()
la_avg_cf10

In [ ]:
la_avg_cf10.loc[la_avg_cf10['layer_id'] == 229, 'layer_id'] = 115

plot

In [ ]:
la_avg_cf10.groupby('category').mean()

cf100

In [ ]:
df_path_cf100 = 'cf100/ww-df'
la_cf100 = delta_merge_layers(df_path_cf100, 'cf100')

In [ ]:
la_cf100 = la_cf100[['layer_id', 'alpha',  'entropy', 'log_norm', 'log_spectral_norm', 'base_name']]
la_cf100

In [ ]:
la_d_cf100 = la_cf100.drop(la_cf100[['alpha',  'entropy', 'log_norm', 'log_spectral_norm']][(la_cf100[['alpha',  'entropy', 'log_norm', 'log_spectral_norm']] == 0).all(axis=1)].index)

In [ ]:
la_dif_cf100 = compute_pairwise_differences(la_d_cf100)
la_dif_cf100['category'] = la_dif_cf100['base_name'].map(categorize_index)

In [ ]:
la_avg_cf100 = la_dif_cf100.drop(['base_name'], axis=1).groupby(['layer_id', 'category'], as_index=False).mean()
la_avg_cf100

In [ ]:
la_avg_cf100.loc[la_avg_cf100['layer_id'] == 229, 'layer_id'] = 115

In [ ]:
la_avg_cf100.groupby('category').mean()

cars

In [ ]:
df_path_car = 'car/ww-df'
la_car = delta_merge_layers(df_path_car, 'int')

la_car = la_car[['layer_id', 'alpha',  'entropy', 'log_norm', 'log_spectral_norm', 'base_name']]
la_car

In [ ]:
la_d_car = la_car.drop(la_car[['alpha',  'entropy', 'log_norm', 'log_spectral_norm']][(la_car[['alpha',  'entropy', 'log_norm', 'log_spectral_norm']] == 0).all(axis=1)].index)

la_dif_car = compute_pairwise_differences(la_d_car)
la_dif_car['category'] = la_dif_car['base_name'].map(categorize_index)

la_dif_car


In [ ]:
la_avg_car = la_dif_car.drop(['base_name'], axis=1).groupby(['layer_id', 'category'], as_index=False).mean()
la_avg_car

In [ ]:
la_avg_car.loc[la_avg_car['layer_id'] == 229, 'layer_id'] = 115

In [ ]:
la_avg_car.groupby('category').mean()

Dom

In [ ]:
df_path_dom = 'domain/ww-df'
la_dom = delta_merge_layers(df_path_dom, 'dom')

la_dom = la_dom[['layer_id', 'alpha',  'entropy', 'log_norm', 'log_spectral_norm', 'base_name']]
la_dom

In [ ]:
la_d_dom = la_dom.drop(la_dom[['alpha',  'entropy', 'log_norm', 'log_spectral_norm']][(la_dom[['alpha',  'entropy', 'log_norm', 'log_spectral_norm']] == 0).all(axis=1)].index)

la_dif_dom = compute_pairwise_differences(la_d_dom)
la_dif_dom['category'] = la_dif_dom['base_name'].map(categorize_index)


la_avg_dom = la_dif_dom.drop(['base_name'], axis=1).groupby(['layer_id', 'category'], as_index=False).mean()
la_avg_dom

In [ ]:
la_avg_dom.loc[la_avg_dom['layer_id'] == 229, 'layer_id'] = 115

In [ ]:
la_avg_dom.groupby('category').mean()

Cross ALL

In [ ]:
la_all = (la_avg_cf10.iloc[:, 2:] + la_avg_cf100.iloc[:, 2:] + la_avg_car.iloc[:, 2:] + la_avg_dom.iloc[:, 2:])/4

In [ ]:
la_all['layer_id'] = la_avg_cf10['layer_id']
la_all['category'] = la_avg_cf10['category']

In [ ]:
la_all.iloc[:, [0, 1, 2, 3, 5]].groupby('category').mean()

In [ ]:
# Create the DataFrame
data = {
    'alpha_diff': [-0.079965, -0.018030, -0.200091],
    'entropy_diff': [-0.000002, 0.000006, 0.001034],
    'log_norm_diff': [-0.000098, -0.000197, -0.049369],
    'log_spectral_norm_diff': [-0.000231, -0.000527, -0.055900]
}
df = pd.DataFrame(data, index=['div', 'dp', 'wd'])

# Set up subplots: 1 row, 5 columns
fig, axes = plt.subplots(1, 4, figsize=(18, 4), sharey=False)

# Plot each metric separately
for i, col in enumerate(df.columns):
    ax = axes[i]
    colors = ['darkblue' if val < 0 else 'skyblue' for val in df[col]]
    
    df[col].plot(kind='bar', ax=ax, color=colors, edgecolor='black')
    ax.set_title(col, fontsize=12)
    ax.set_xticks(range(len(df.index)))
    ax.set_xticklabels(df.index, rotation=45, fontsize=10)
    ax.tick_params(axis='y', labelsize=9)
    ax.grid(axis='y', linestyle='--', alpha=0.5)


plt.tight_layout()  

plt.show()


In [ ]:
la_all.loc[la_all['layer_id'] == 229, 'layer_id'] = 115

In [ ]:
la_all[la_all['layer_id']==115].iloc[:,:-2].to_dict(orient='list')

In [ ]:
# Create the DataFrame
data = {
'alpha_diff': [-2.9730697105693196, -0.603896775637893, -2.407761239633577],
 'entropy_diff': [0.00026470140300282886,
  0.0004323181034272816,
  -0.00019388513578663666],
 'log_norm_diff': [-0.019426871401561442,
  -0.008695592273255254,
  -0.0058425671961644995],
 'log_spectral_norm_diff': [-0.01904593698137189,
  -0.02184169612409343,
  -0.014942487038449503]
}
df = pd.DataFrame(data, index=['div', 'dp', 'wd'])

# Set up subplots: 1 row, 5 columns
fig, axes = plt.subplots(1, 4, figsize=(18, 4), sharey=False)

# Plot each metric separately
for i, col in enumerate(df.columns):
    ax = axes[i]
    colors = ['darkblue' if val < 0 else 'skyblue' for val in df[col]]
    
    df[col].plot(kind='bar', ax=ax, color=colors, edgecolor='black')
    ax.set_title(col, fontsize=12)
    ax.set_xticks(range(len(df.index)))
    ax.set_xticklabels(df.index, rotation=45, fontsize=10)
    ax.tick_params(axis='y', labelsize=9)
    ax.grid(axis='y', linestyle='--', alpha=0.5)


plt.tight_layout()  

plt.show()


In [ ]:
variables = ['alpha_diff', 'entropy_diff', 'log_norm_diff', 'log_spectral_norm_diff']

# Pivot the data to have categories as columns
pivot_df = la_all.pivot(index='layer_id', columns='category', values=variables)

# Compute Pearson correlation between categories for each variable
for var in variables:
    corr_matrix = pivot_df[var].corr(method='pearson')
    print(f'Pearson Correlation for {var}:\n{corr_matrix}\n')


In [ ]:
import itertools
from scipy.stats import f_oneway, ttest_ind

categories = ['div', 'dp', 'wd']

for var in variables:
    print(f'Variable: {var}')
    # Create a dictionary to hold series data for each category
    series_dict = {}
    for cat in categories:
        series = pivot_df[var][cat].dropna().values
        series_dict[cat] = series
    
    # ANOVA
    f_stat, p_value = f_oneway(*[series_dict[cat] for cat in categories])
    print(f'ANOVA: F-statistic={f_stat:.4f}, p-value={p_value:.4f}')
    
    # Pairwise t-tests
    for cat1, cat2 in itertools.combinations(categories, 2):
        t_stat, p_value = ttest_ind(series_dict[cat1], series_dict[cat2])
        print(f't-test between {cat1} and {cat2}: t-statistic={t_stat:.4f}, p-value={p_value:.4f}')
    print('-' * 50)

In [ ]:
# zoom-in
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, mark_inset

# Main plot
fig, ax = plt.subplots(figsize=(10, 6))
sns.scatterplot(
    data=la_all,
    x='layer_id',
    y='alpha_diff',
    hue='category',
    style='category',
    s=100,
    alpha=0.8,
    palette='Set2',
    ax=ax
)

# Plot formatting
ax.set_xlabel('Layer Index', fontsize=14)
ax.set_ylabel('Alpha', fontsize=14)
ax.tick_params(axis='both', labelsize=12)
ax.grid(True)

# Legend
ax.legend(title='Category', loc='lower right', fontsize=12, title_fontsize=12)

# ✅ Use absolute size + custom bottom-middle anchor
axins = inset_axes(
    ax,
    width=4, height=2,  # inches
    bbox_to_anchor=(0.25, 0.01, 0.5, 0.5),  # (x0, y0, width, height) in axes coords
    bbox_transform=ax.transAxes,
    loc='lower left'
)

# Zoomed-in scatterplot
sns.scatterplot(
    data=la_all,
    x='layer_id',
    y='alpha_diff',
    hue='category',
    style='category',
    s=70,
    alpha=0.8,
    palette='Set2',
    ax=axins,
    legend=False
)

title = (
    "ANOVA: F=0.84, p=0.43 | "
    "t(div-dp): t=-0.77, p=0.45 | "
    "t(div-wd): t=0.69, p=0.49 | "
    "t(dp-wd): t=1.17, p=0.25"
)
plt.suptitle(title, fontsize=12)


# Zoom-in limits and line
axins.set_ylim(-0.08, 0.08)
axins.set_xlim(la_all['layer_id'].min()-5, la_all['layer_id'].max()+5)
axins.axhline(0, color='black', linestyle='--', linewidth=1)
axins.set_xticklabels([])
axins.set_yticklabels([])
axins.set_xlabel('')
axins.set_ylabel('')

# Optional: mark_inset (comment if visually confusing at bottom)
mark_inset(ax, axins, loc1=1, loc2=2, fc="none", ec="gray")

plt.tight_layout()

plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
sns.scatterplot(
    data=la_all,
    x='layer_id',
    y='entropy_diff',
    hue='category',
    style='category',
    s=100,
    alpha=0.8,
    palette='Set2',
    ax=ax
)

# Plot formatting
ax.set_xlabel('Layer Index', fontsize=14)
ax.set_ylabel('Entropy', fontsize=14)
ax.tick_params(axis='both', labelsize=12)
ax.grid(True)

# Legend
ax.legend(title='Category', loc='upper right', fontsize=12, title_fontsize=12)

# ✅ Use absolute size + custom bottom-middle anchor
axins = inset_axes(
    ax,
    width=4, height=2,  # inches
    bbox_to_anchor=(0.25, 0.5, 0.5, 0.5),  # (x0, y0, width, height) in axes coords
    bbox_transform=ax.transAxes,
    loc='upper left'
)

# Zoomed-in scatterplot
sns.scatterplot(
    data=la_all,
    x='layer_id',
    y='entropy_diff',
    hue='category',
    style='category',
    s=70,
    alpha=0.8,
    palette='Set2',
    ax=axins,
    legend=False
)

title = (
    "ANOVA: F=1.10, p=0.34 | "
    "t(div-dp): t=-0.41, p=0.68 | "
    "t(div-wd): t=-1.05, p=0.30 | "
    "t(dp-wd): t=-1.05, p=0.30"
)
plt.suptitle(title, fontsize=12)

# Zoom-in limits and line
axins.set_ylim(-0.0001, 0.0001)
axins.set_xlim(la_all['layer_id'].min()-5, la_all['layer_id'].max()+5)
axins.axhline(0, color='black', linestyle='--', linewidth=1)
axins.set_xticklabels([])
axins.set_yticklabels([])
axins.set_xlabel('')
axins.set_ylabel('')

# Optional: mark_inset (comment if visually confusing at bottom)
mark_inset(ax, axins, loc1=1, loc2=2, fc="none", ec="gray")

plt.tight_layout()

plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
sns.lineplot(
    data=la_all,
    x='layer_id',
    y='log_norm_diff',
    hue='category',
    style='category',
    markers=True,
    dashes=False,
    palette='Set2',
    ax=ax
)

# Plot formatting
ax.set_xlabel('Layer Index', fontsize=14)
ax.set_ylabel('Log Frobenius Norm', fontsize=14)
ax.tick_params(axis='both', labelsize=12)
ax.grid(True)

# Legend
ax.legend(title='Category', loc='right', fontsize=12, title_fontsize=12)

# ✅ Use absolute size + custom bottom-middle anchor
axins = inset_axes(
    ax,
    width=4, height=2,  # inches
    bbox_to_anchor=(0.25, 0.15, 0.3, 0.3), # (x0, y0, width, height) in axes coords
    bbox_transform=ax.transAxes,
    loc='upper left'
)

# Zoomed-in scatterplot
sns.lineplot(
    data=la_all,
    x='layer_id',
    y='log_norm_diff',
    hue='category',
    style='category',
    markers=True,
    dashes=False,
    palette='Set2',
    ax=axins,
    legend=False
)

title =  ("ANOVA: F=55.63, p<0.001 | "
    "t(div-dp): t=0.17, p=0.86 | "
    "t(div-wd): t=7.47, p<0.001 | "
    "t(dp-wd): t=7.47, p<0.001")
plt.suptitle(title, fontsize=12)

# Zoom-in limits and line
axins.set_ylim(-0.0003, 0.001)
axins.set_xlim(la_all['layer_id'].min()-5, la_all['layer_id'].max()+5)
axins.axhline(0, color='black', linestyle='--', linewidth=1)
axins.set_xticklabels([])
axins.set_yticklabels([])
axins.set_xlabel('')
axins.set_ylabel('')

# Optional: mark_inset (comment if visually confusing at bottom)
mark_inset(ax, axins, loc1=1, loc2=2, fc="none", ec="gray")

plt.tight_layout()

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
sns.lineplot(
    data=la_all,
    x='layer_id',
    y='log_spectral_norm_diff',
    hue='category',
    style='category',
    markers=True,
    dashes=False,
    palette='Set2',
    ax=ax
)

# Plot formatting
ax.set_xlabel('Layer Index', fontsize=14)
ax.set_ylabel('Log Spectral Norm', fontsize=14)
ax.tick_params(axis='both', labelsize=12)
ax.grid(True)

# Legend
ax.legend(title='Category', loc='right', fontsize=12, title_fontsize=12)

# ✅ Use absolute size + custom bottom-middle anchor
axins = inset_axes(
    ax,
    width=4, height=2,  # inches
    bbox_to_anchor=(0.25, 0.15, 0.3, 0.3), # (x0, y0, width, height) in axes coords
    bbox_transform=ax.transAxes,
    loc='upper left'
)

# Zoomed-in scatterplot
sns.lineplot(
    data=la_all,
    x='layer_id',
    y='log_spectral_norm_diff',
    hue='category',
    style='category',
    markers=True,
    dashes=False,
    palette='Set2',
    ax=axins,
    legend=False
)

title =  (
    "ANOVA: F=32.90, p<0.001 | "
    "t(div-dp): t=0.37, p=0.71 | "
    "t(div-wd): t=5.76, p<0.001 | "
    "t(dp-wd): t=5.73, p<0.001"
    )
plt.suptitle(title, fontsize=12)

# Zoom-in limits and line
axins.set_ylim(-0.003, 0.005)
axins.set_xlim(la_all['layer_id'].min()-5, la_all['layer_id'].max()+5)
axins.axhline(0, color='black', linestyle='--', linewidth=1)
axins.set_xticklabels([])
axins.set_yticklabels([])
axins.set_xlabel('')
axins.set_ylabel('')

# Optional: mark_inset (comment if visually confusing at bottom)
mark_inset(ax, axins, loc1=1, loc2=2, fc="none", ec="gray")

plt.tight_layout()
plt.show()